![MIoT_GDPI](img/MIOT_GDPI_header.png)

# Unidad 01 - Introducción al stream learning

El objetivo principal de esta práctica es que os familiaricéis con los conceptos más básicos del Aprendizaje *Online* (aka *stream learning* o *incremental learning*). Se desarrollarán brevemente algunas definiciones para entender el contexto, la problemática y las diferencias con el aprendizaje automático tradicional y se introducirá en el uso de la librería [River](https://riverml.xyz/) que permite desarrollar modelos incrementales de forma intuitiva empleando Python.  


La mayor parte del contenido de este Notebook se dedica a explicar los aspectos fundamentales del aprendizaje incremental, apoyándose en ejemplos concretos que ilustran su aplicación a un problema real. Es crucial que dediquéis tiempo a leer y comprender el material, en lugar de simplemente ejecutar el código. Os invitamos a experimentar modificando y variando el código proporcionado para que podáis explorar las distintas opciones y profundizar en su funcionamiento.



**Importante**: El Notebook contiene varios ejercicios sencillos. Deberéis desarrollarlos durante la clase y enviarlos por el aula virtual del curso, en la tarea correspondiente.



## Referencias útiles para la práctica
1. API Pandas: [https://pandas.pydata.org/docs/reference/index.html](https://pandas.pydata.org/docs/reference/index.html)
2. Api River: [https://riverml.xyz](https://riverml.xyz)
3. Api Scikit-Learn: [https://scikit-learn.org/stable/api/index.html](https://scikit-learn.org/stable/api/index.html)
4. Bahri, M., Bifet, A., Gama, J., Gomes, H. M., & Maniu, S. (2021). [Data stream analysis: Foundations, major tasks and tools](https://doi.org/10.1002/widm.1405). Wiley Interdisciplinary Reviews: Data Mining and Knowledge Discovery, 11(3), e1405.
5. Gomes, H. M., Read, J., Bifet, A., Barddal, J. P., & Gama, J. (2019). [Machine learning for streaming data: state of the art, challenges, and opportunities](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://kdd.org/exploration_files/3._CR_7._Machine_learning_for_streaming_data_state_of_the_art-Final.pdf). ACM SIGKDD Explorations Newsletter, 21(2), 6-22.

## 1. Introducción al concepto de Stream Learning

El tiempo es una característica clave de la mayoría de las actividades humanas de interés.  Ya sea la evolución temporal del mercado de valores, la dinámica de las redes sociales, la entrada sensorial a los robots o en la fabricación dentro de las líneas de producción industriales, el flujo del tiempo es un factor importante para la comprensión del sistema y la toma de decisiones. 

A través del paso del tiempo, la información se actualiza y evoluciona y, ante estos cambios, nos adaptamos a la nueva información, interpretamos su significado y modificamos, si es necesario, nuestras suposiciones subyacentes. Por lo tanto, los sistemas de IA/ML en tiempo real deben ser capaces de adaptarse de manera similar para ser efectivos.

Los datos continuos basados en el tiempo se consideran *streams* (flujos). Los algoritmos de aprendizaje de IA que pueden adaptarse a estos flujos de manera "incremental" han dado lugar a un nuevo paradigma, conocido como *stream learning*, *incremental learning* u *online learning*. 

Debido a que los sistemas reales están caracterizados por fenómenos que pueden ocurrir en diferentes escalas temporales (a corto o a largo plazo), estos sistemas IA pueden analizar información también en múltiples escalas. Por ejemplo, el comportamiento a corto plazo podría ser las fluctuaciones de 
humedad en el desarrollo de un producto concreto en una línea industrial a lo largo de un día, mientras que el comportamiento a largo plazo podría considerarse el promedio de humedades a lo largo de un año en el que la influencia de las estaciones meteorológicas es relevante para la producción.

¿Cómo podría un sistema de IA proporcionar predicciones precisas teniendo en cuenta escalas de tiempo radicalmente diferentes? Más importante aún, ¿Cómo podría un sistema de IA comprender que el flujo de datos entrante está evolucionando continuamente y que los fenómenos subyacentes pueden deberse a nuevas tendencias que no estaban presentes en los datos de entrenamiento originales? Estas son precisamente las preguntas que abordan los sistemas de aprendizaje incremental (aka *online learning* o *stream learning*).

###  1.1. Comparativa con el ML clásico

Para comprender mejor el aprendizaje *online*, es útil compararlo con el enfoque tradicional de aprendizaje automático (*offline learning* o *batch learning*). Como ejemplo, consideremos una señal con dependencias temporales como la de un sensor de temperatura, que consta de dos elementos: el tiempo y la amplitud correspondiente (un voltaje escalado, correlacionado con la temperatura).

Un algoritmo clásico de aprendizaje automático podría obtener información en múltiples escalas temporales analizando primero la relación a corto plazo de la señal; esto se lograría mediante el uso de una ventana deslizante y analizando las características de la señal dentro de cada una de esas ventanas. Con ventanas más grandes y datos históricos, el algoritmo clásico podría estudiar estructuras temporales más largas.

De esta manera, un enfoque tradicional de aprendizaje automático generalmente construiría un conjunto de datos para entrenamiento y evaluación. Este conjunto de datos incluiría varias secciones de la señal que corresponden a características y salidas en diferentes escalas temporales, proporcionando cierto grado de predictibilidad tanto para eventos futuros a corto como a largo plazo. Sin embargo, las limitaciones de este enfoque se hacen evidentes de inmediato. En primer lugar, las predicciones se vuelven más complejas porque el sistema debe distinguir entre las diferentes escalas temporales (y los límites entre ellas). Si el sistema debe hacer una predicción basada en un solo punto de datos de entrada, ¿cómo se incorpora la información multiescala en el algoritmo de predicción? En otras palabras, la evolución a largo plazo de la entrada no es evidente en un solo punto de datos, lo que dificulta capturar predicciones a largo plazo sobre cambios subyacentes.

Como resultado, los enfoques clásicos de aprendizaje automático para la predicción multitemporal suelen ser *ad-hoc* y dependen de modelos diseñados específicamente para adaptarse a la evolución de los fenómenos subyacentes, lo que hace que el entrenamiento previo sea incompleto o inexacto. Por lo tanto, el enfoque común para abordar este problema es reentrenar y volver a desplegar el modelo de aprendizaje automático de manera periódica. En algunos sistemas, estas actualizaciones del modelo se realizan en escalas de tiempo cortas, como minutos o incluso segundos.

### 1.2. El nuevo paradigma: 

Para abordar de manera efectiva los flujos temporales y los problemas  descritos anteriormente, surge el aprendizaje incremental (aka *online learning o *stream learning*), es decir, sistemas ML específicamente diseñados para manejar flujos (*streams*) de datos continuos y, a menudo, infinitos. Con cada nueva observación que estos sistemas reciben ajustan el modelo de predicción, en lugar de utilizar conjuntos de datos estáticos (ej. datasets de entrenamiento) o grandes lotes (*batches*).

Si bien la diferencia entre el aprendizaje *online* y el aprendizaje clásico puede parecer sutil, existen varias diferencias técnicas  que deben ser consideradas y que serán presentadas a lo largo de las unidades de trabajo junto con  lós métodos necesarios para implementarlas.

Antes de comenzar, proporcionaremos algunas definiciones formales y conceptos básicos.


## 2. Flujos de datos (*Data Streams*)

En el núcleo del aprendizaje *online* se encuentra el término “flujo de datos” (*data stream*), que se define como la recopilación secuencial y continua de elementos individuales coherentes (observaciones). En el contexto de la información procesada por un algoritmo o sistema de aprendizaje automático, este flujo de datos normalmente consiste en datos (o metadatos) que pueden utilizarse para generar un conjunto de características medidas simultáneamente en el momento de su recepción. Estos datos suelen ser dependientes del tiempo o estar correlacionados temporalmente mediante una marca de tiempo. Por ejemplo, en una línea de producción podríamos considerar cada observación  como las medidas tomadas por los diferentes sensores desplegados por la fábrica para cada instante de tiempo.

Una observación (o muestra) se define como el conjunto de características medidas en un instante específico. Estas muestras pueden tener una estructura de datos estable, es decir, en cada muestra están disponibles los mismos atributos para ser incluidos en la medición o modelo, o bien, la estructura de datos de la muestra puede ser “dinámica” o flexible, donde los atributos aparecen intermitentemente a lo largo del tiempo. La flexibilidad es una característica que no puede ser tratada por el paradigma tradicional de aprendizaje automático pero sí con el incremental.



### 2.1. Flujos de datos reactivos y proactivos


Los flujos de datos (*streams*) pueden clasificarse en dos tipos: reactivos y proactivos, dependiendo de su relación con el usuario.

Los **flujos de datos reactivos** son aquellos en los que **el sistema "recibe" datos de un productor**. Un ejemplo típico es un sitio web que genera flujos de datos, como el servidor de datos de 'X' (*Twitter*) o también el sistema de producción de una factoría. En estos casos, la forma en que se produce el flujo de datos no está bajo el control del sistema de procesamiento; un sistema aprendizaje *online* simplemente actúa como receptor y no tiene influencia ni control más allá de recibir los datos y actuar en consecuencia. En este sentido, el sistema, simplemente,  "reacciona" a la entrada de datos.

Los **flujos de datos proactivos** son aquellos en los que **el sistema controla cómo y cuándo se obtiene los datos desde el *stream***. Por ejemplo, se puede controlar el momento y la manera en que se leen los datos de un archivo. El sistema podría decidir a qué velocidad leer los datos, en qué orden, etc.



## 3. Procesamiento *online*

El concepto de “en línea” (*online*) se refiere al procesamiento de un flujo de datos (*stream*) observación por observación a medida que se van generando. Debido a ciertas confusiones con el término “*online education*”, suele emplearse el término *incremental learning* para evitar malos entendidos. En cualquier caso, el término se refiere al entrenamiento de un modelo mediante la incorporación de una muestra a la vez y la actualización de los pesos del modelo de manera continua.  
De este modo, el aprendizaje incremental es significativamente diferente de los enfoques tradicionales de aprendizaje automático, que procesan datos en lotes (*batches*). 
Este cambio de metodología implica nuevos requisitos computacionales y técnicos. Por ejemplo, dado que en el procesamiento incremental las observaciones se gestionan de una en una, no se obtiene ningún beneficio de la vectorización y bibliotecas como NumPy y PyTorch solo generan un coste computacional adicional e innecesario.

Es importante destacar que los modelos de aprendizaje *online*  son dinámicos y con estado. Este tipo de aprendizaje automático representa un nuevo paradigma en ML y, como cualquier otro método técnico, tiene ventajas y desventajas cuando se aplica a problemas del mundo real.


##  4. Proceso de entrenamiento

En la mayoría de las aplicaciones del mundo real los flujos de datos son reactivos pero, sin embargo, es habitual que para construir, entrenar y evaluar el rendimiento de un modelo de ML *online* , se genere un conjunto de datos que simule el comportamiento del flujo de datos (*data stream* proactivo). Como veremos, la librería River gestiona esto de manera elegante al convertir el conjunto de datos en un objeto [generador](https://wiki.python.org/moin/Generators) de Python. Una vez desarrollado el modelo, el flujo de datos en tiempo real puede reemplazar sin inconvenientes la versión de desarrollo. 
Es importante destacar que los modelos de ML *online* no requieren ser entrenados de esta forma, ya que es posible desplegarlos directamente y aprender el tiempo real, no obstante, es habitual desplegarlos con un mínimo de entrenamiento para evitar grandes errores al comienzo.

Hay una diferencia fundamental en la forma en la que el aprendizaje automático tradicional y el aprendizaje incremental entrenan los modelos. En el aprendizaje tradicional el conjunto de datos histórico se divide en entrenamiento y evaluación. El conjunto de evaluación se reserva y no se emplea hasta que el modelo está entrenado pero, sin embargo, en el aprendizaje *online*, se emplean todos los datos para las dos etapas (entrenamiento y evaluación).
Así, cuando llega una nueva observación, la aproximación *online* la usa para evaluar el modelo y luego ajusta los pesos del modelo (entrenamiento) en consecuencia. Este proceso se repite continuamente para todo los datos del *stream*, por lo que el orden temporal de las observaciones es crucial en el desarrollo del modelo. En contraste, en el aprendizaje tradicional, los datos suelen dividirse y mezclarse aleatoriamente para reducir correlaciones estadísticas. 


El orden temporal garantiza que el modelo se ajuste a la realidad de los datos en cada instante de tiempo y es clave para descubrir la estructura causal subyacente de la información y realizar los ajustes necesarios para realizar predicciones más precisas en el futuro.



## 5. Concept Drift

En el contexto de los flujos de datos, la razón principal por la que el aprendizaje automático tradicional falla (o al menos no es tan adecuado y requiere procesos complicados de reentrenamiento) es el fenómeno conocido como *concept drift*. Este término se refiere a la situación en la que los datos cambian o evolucionan con el tiempo, haciendo que los modelos entrenados con datos pasados queden obsoletos. 

Si bien, como veremos, el *concept drift* también es un reto para las aproximaciones incrementales, la ventaja de estas es que los ajustes en los modelos se realizan de manera continua intentando adaptarse a dichos cambios. 
No obstante, este sigue siendo un campo de investigación activo, que exploraremos con mayor profundidad durante el curso




## 6. Librerías para trabajar con Aprendizaje Automático incremental

Dado que el ML *online* es un paradigma relativamente nuevo, todavía existen pocas implementaciones y pocas de estas están suficientemente maduras. 
A continuación, se presenta un resumen de algunas de las principales librerías que podéis encontrar:

* **[Apache SAMOA](https://incubator.apache.org/projects/samoa.html)**, un proyecto para realizar análisis y minería de datos en flujos de datos. Cuenta con módulos específicos para aprendizaje automático. No obstante, el proyecto no se ha actualizado desde 2020 y sigue en la fase de incubación de la Fundación Apache (AP). ~~Además, se rumorea que la AP tiene planes de abandonar su desarrollo.~~ Finalmente, el proyecto SAMOA fue retirado.

* **[MOA](https://moa.cms.waikato.ac.nz/)**, escrito en Java,  desarrollado por los mismos autores del proyecto WEKA y con una estrecha relación con dicho proyecto. Incluye una colección de algoritmos de aprendizaje automático (clasificación, regresión, clustering, detección de valores atípicos, detección de concept drift y sistemas de recomendación) y herramientas para la evaluación. Su uso está principalmente limitado a la interfaz que proporciona o a la implementación de extensiones enfocadas a la integración con el resto del ecosistema.

* **[Vowpal wabbit](https://vowpalwabbit.org/)**, una librería de Python soportada por Microsoft. Ofrece herramientas enfocadas al ML *online* pero con un énfasis particular en problemas de aprendizaje por refuerzo. Una desventaja significativa es que requiere un formato de entrada de datos específico (poco intuitivo), lo que limita en gran medida su usabilidad.

* **[River](https://riverml.xyz/)**, una librería de Python enfocada en al aprendizaje incremental, que ha adoptado un enfoque más general (y *pythonic*) en comparación con *Vowpal Wabbit*. En esta librería, el número de modelos es similar a los de scikit-learn, y además ofrece herramientas para interactuar con el ecosistema de scikit-learn a través de *wrappers*. También permite desarrollar soluciones de aprendizaje por refuerzo. Su principal ventaja es que resulta muy simple e intuitivo para los nuevos desarrolladores, sobre todo si tienen experiencia en el *pipeline* de scikit-learn. Además permite trabajar con los tipos de datos más comunes como, por ejemplo, [Pandas](https://pandas.pydata.org/) DataFrames.

Durante las prácticas enfocadas al aprendizaje incremental, emplearemos River como librería para el desarrollo de soluciones.
Mediante el uso de River, describiremos diferentes casos de uso y abordaremos muchos de los problemas comunes en el aprendizaje *online* o incremental.

El resto de este Notebook explorará algunos ejemplos comunes empleando River.

- El repositorio de código de River está disponible en el siguiente enlace: [River Repository](https://github.com/online-ml/river/tree/main/river)
- La referencia de la API de River está disponible en el siguiente enlace: [River API](https://riverml.xyz/latest/api/overview/)

 



## 7. Aprendizaje online supervisado: Clasificación binaria

Este es quizás el ejemplo más elemental del aprendizaje automático, y proporciona un buen punto de partida para discutir el aprendizaje *online*. En este caso, el modelo tiene una única salida que describe a cuál de las dos clases pertenece la muestra.

Pasos: 

1. **Instalar la librería** (si no está instalada, se llama al comando pip).  


In [ ]:
#Es necesario al menos una versión de  Python >3.8
#Si tenéis el entorno virtual correctamente creado, esto no se ejecutará
try:
    import river
except ImportError as err:
    !pip install river

    
# Este librería genera un print mejorado
# https://rich.readthedocs.io/en/stable/introduction.html
#Si tenéis el entorno virtual correctamente creado, esto no se ejecutará
try:
    from rich import print
except ImportError as err:
    !pip install rich
    from rich import print

2. **Importar el conjunto de datos**: para este ejemplo, hemos seleccionado un [conjunto de datos](https://riverml.xyz/latest/api/datasets/CreditCard/) que pertenece a los datasets de ejemplo de River, por su facilidad para cargarlo y procesarlo. Los datos representan operaciones con tarjetas de crédito y el objetivo es detectar fraudes bancarios.

In [ ]:
from rich import print
from river import datasets #Datasets de River


dataset = datasets.CreditCard()
print(f"El objeto contiene información sobre el datasets como, por ejemplo, el número de ejemplos y las características.")
print(dataset)

In [ ]:
# Observemos el primer ejemplo
#Creamos un iterador para recorrer el dataset y obtenemos la primera observación
#Cada observación está compuesta de sus características y su etiqueta asociada
sample, target = next(iter(dataset)) 
print("Características:")
print(sample)
print("Etiqueta:")
print(target)

Trabajar con clases desbalanceadas es una situación bastante común en el aprendizaje *online* (ej. detección de fraude o clasificación de spam).
Como observamos en la descripción, el conjunto de datos CreditCard está claramente desbalanceado. Nosotros también podríamos calcular fácilmente el porcentaje de representación de los datos dentro de cada clase:

In [ ]:
import collections #librería de Python

#Genera un diccionario con las etiquetas y el número de observaciones que las contiene
counts = collections.Counter(target for _, target in dataset)

for label, count in counts.items():
    print(f'{label}: {count} ({count / sum(counts.values()):.2%})')


En este ejemplo, nos centraremos en cómo procesar los datos empleando el paradigma incremental y no abordaremos el problema del desbalanceo pero, no obstante, existen diferentes aproximaciones para gestionar este tipo de problemas y mejorar el rendimiento de los modelos.

3. **Desarrollar un modelo para clasificación binaria**: Emplearemos un modelo básico ([logisticRegression](https://riverml.xyz/latest/api/linear-model/LogisticRegression/)) a modo de ejemplo.

In [ ]:
from river import linear_model

model = linear_model.LogisticRegression()

Sin entrenar adecuadamente el modelo, si procesamos una observación obtendremos como resultado que las probabilidades para cada clase son exactamente las mismas. La función `predict_proba_one` nos permite analizar dichas probabilidades.

In [ ]:
print(model.predict_proba_one(sample))

Para cada clase, tenemos un clasificador aleatorio sin ningún conocimiento. Aquí es donde el método difiere del aprendizaje automático tradicional. La misma muestra que se usó para probar, también se usará para ajustar el modelo. Debéis tener en cuenta que las métricas de rendimiento deben calcularse antes de ajustar el modelo para que sean justas.

4. **Entrenar el modelo con la observación actual**: el modelo se entrenará con los datos de la observación actual y su salida real, que deberemos de obtener de algún modo. Sin los datos reales, no podríamos ajustar el modelo. En una planta industrial es habitual predecir variables cuyos valores reales pueden ser medidos posteriormente en laboratorio o por sensórica.


In [ ]:
model.learn_one(sample, target)

Si probamos nuevamente la misma observación, veremos que las probabilidades han cambiado.

In [ ]:
#En este ejemplo estamos reutilizando una observación empleada para ajustar los pesos para evaluar el modelo
#Esto no es ni correcto ni justo y se realiza solo con fines académicos. 
#La secuencia adecuada para cada muestra sería: predicción - evaluación - entrenamiento.

print(model.predict_proba_one(sample))




Para obtener la etiqueta de salida sin las probabilidades asociadas, se puede emplear la función: <code>predict_one()</code>.

In [ ]:
print(model.predict_one(sample))

Veamos ahora un proceso completo en el que integramos en un bucle la predicción, la evaluación y el ajuste de los modelos. 
En este caso, usamos para evaluar el modelo la métrica estándar que consiste en el área bajo la curva [ROC](https://riverml.xyz/latest/api/metrics/ROCAUC/). No obstante, podríamos haber seleccionado cualquier otra métrica de las disponibles en River



In [ ]:
from river import metrics

model = linear_model.LogisticRegression()
metric = metrics.ROCAUC()

for sample, target in dataset:
    prediction = model.predict_proba_one(sample)
    metric.update(target, prediction)
    model.learn_one(sample, target)
   

print(metric)

Una aproximación común y sencilla para mejorar el rendimiento del modelo es escalar los datos. River proporciona múltiples operaciones de preprocesamiento, incluyendo métodos para el escalado como, por ejemplo, la estandarización a través de [preprocessing.StandardScaler](https://riverml.xyz/latest/api/preprocessing/StandardScaler/). 

River proporciona una interfaz y sintaxis similar a `scikit-learn`, lo que simplificará su uso para desarrolladores que tengan conocimientos previos en dicha librería. En el siguiente ejemplo vemos como en River se pueden emplear e integrar *pipelines* de forma similar a `scikit-learn`.


En este ejemplo, no escribiremos un bucle explícito para desarrollar el proceso de *entrenamiento-evaluación-actualización* porque la función <code> evaluate.progressive_val_score</code> lo realizará automáticamente por nosotros.


In [ ]:
from river import evaluate, compose, preprocessing

#Ahora el "modelo" se compone de un paso de preprocesado + el propio modelo
model = compose.Pipeline(
    preprocessing.StandardScaler(),
    linear_model.LogisticRegression()
)

print(model)

metric = metrics.ROCAUC()
evaluate.progressive_val_score(dataset, model, metric)

#progressive_val_score es equivalente a:
#for sample, target in dataset:
#    prediction = model.predict_proba_one(sample)
#    metric.update(target, prediction)
#    model.learn_one(sample, target)


### Ejercicio 1

La métrica seleccionada para evaluar un modelo que trabaja con clases desbalanceadas es crítica. Puedes intentar evaluar el modelo usando la métrica de Precisión ([Accuracy](https://riverml.xyz/latest/api/metrics/Accuracy/)) (está incluida en la API de River). Verás que obtienes métricas impresionantes sin escalar los datos. Un modelo que siempre prediga la clase "no fraude" obtendrá una muy alta precisión, ya que es la clase mayoritaria. **Crea un *pipeline* empleando la métrica de precisión (*accuracy*)**



In [ ]:
#Crea un pipeline empleando la métrica de precisión (accuracy)





### Ejercicio 2


El coeficiente Kappa es una métrica muy útil para evaluar modelos con clases desbalanceadas. Este coeficiente mide la concordancia entre la etiqueta deseada y la etiqueta proporcionada por la salida del modelo, excluyendo la probabilidad de concordancia por azar. Esta métrica se considera generalmente más robusta que la precisión y su valor suele ser más bajo. El coeficiente [Kappa de Cohen](https://riverml.xyz/latest/api/metrics/CohenKappa/) también está disponible en River.



**Evalúa el modelo usando la métrica *Kappa*. Comprueba cómo la métrica varía empleando la estandarización**



In [ ]:
#Crea un pipeline empleando la métrica Kappa






### Ejercicio 3
¿Qué os parece si probamos con otro modelo? Repite el *pipeline* empleando otro modelo diferente. Por ejemplo, podéis emplear el [Perceptron](https://riverml.xyz/latest/api/linear-model/Perceptron/)

In [ ]:
#Crea otro pipeline empleando otro modelo de aprendizaje automático





## 7. Aprendizaje online supervisado: Clasificación multiclase

El uso del aprendizaje *online* para realizar clasificación multiclase es el siguiente paso en complejidad que podemos considerar.
Los pasos son similares a los empleados en la clasificación binaria, con la diferencia de que es necesario modificar las funciones de pérdida para tener en cuenta las múltiples etiquetas salidas. En el siguiente ejemplo, utilizamos otro [conjunto de datos](https://riverml.xyz/latest/api/datasets/ImageSegments/) de River que representa un conjunto de imágenes con 7 posibles clases.


In [ ]:
from rich import print
from river import datasets

dataset = datasets.ImageSegments()
print(dataset)

Al igual que en la clasificación binaria, el conjunto de datos consiste en una serie de observaciones etiquetadas en formato de tupla (inputs, output). Una diferencia respecto al problema previo con el clasificador binario es la elección del clasificador multiclase. En este caso, empleamos un nuevo método de clasificación algo más complejo llamado [árbol de Hoeffding] (https://riverml.xyz/latest/api/tree/HoeffdingTreeClassifier/). Este clasificador pertenece al módulo "tree" de River. 

En el ejemplo podemos comprobar que, antes de entrenar, si imprimos las probabilidades de cada clase para una observación específica (`predict_proba_one`) obtendremos un diccionario vacío.  La razón es que el modelo aún no ha visto ninguna observación, por lo que no tiene información sobre las "posibles" clases. Si se tratara de un clasificador binario, emitiría una probabilidad del 50% para Verdadero y Falso porque las clases serían implícitas. Sin embargo, en este caso, como estamos realizando una clasificación multiclase,  la salida es nula. En consecuencia, el método `predict_one` inicialmente devolverá `None` porque el modelo aún no ha visto ningún ejemplo etiquetado.

In [ ]:
from river import tree

data_stream = iter(dataset)
sample, target = next(data_stream)

model = tree.HoeffdingTreeClassifier()#modelo "vacío", sin entrenar
print("Probabilidades del modelo sin entrenar:", model.predict_proba_one(sample))

print("Predicción de la clase del modelo sin entrenar:", model.predict_one(sample))


A medida que el modelo analice observaciones con sus respectivas etiquetas (clases), irá  añadiendo dichas clases a las probabilidades del propio modelo. Por ejemplo, una vez procesada la primera observación con su clase correspondiente, el modelo asociará un 100% de probabilidad de que la muestra pertenezca a dicha clase, ya que en ese punto no hay otras opciones posibles (solo se ha analizado una muestra). A medida que más muestras se analicen, más clases se añadirán al modelo.

In [ ]:
print(sample, target)
model.learn_one(sample, target)
print("Probabilidades del modelo entrenado con 1 sola observación:" , model.predict_proba_one(sample))
print("Predicción de la clase del modelo entrenado con 1 sola observación: ", model.predict_one(sample))

Si analizamos una segunda observación, veremos que las probabilidades cambian (estamos añadiendo una nueva etiqueta).

In [ ]:
sample, target = next(data_stream) # Siguiente observación en el stream
print(sample, target)
model.learn_one(sample, target)
print("Probabilidades del modelo entrenado con 2 observaciones:", model.predict_proba_one(sample))
print("Predicción de la clase del modelo entrenado con 2 observaciones: ",model.predict_one(sample))

Este es uno de los puntos clave de los sistemas de clasificación incremental: **los modelos pueden gestionar la aparición de nuevas clases en el flujo de datos**.

Típicamente, las observaciones se usan una vez para hacer una predicción y, cuando se realiza la predicción, la "verdad de tierra" (*ground-truth*) surgirá en algún momento posterior y se podrá emplear para evaluar el modelo y también para ajustarlo (entrenarlo). Este esquema se llama usualmente **validación progresiva**.

Imaginaros en una fábrica suficientemente sensorizada en la que estamos prediciendo un resultado de calidad en la producción de un producto concreto. Nuestro objetivo es adelantarnos y predecir con suficientemente tiempo para poder actuar sobre la configuración pero llegará un momento en que los sensores desplegados realmente puedan "leer" la "verdad de tierra" y con ello ajustar el modelo.

Probemos ahora con todas las observaciones.


In [ ]:
from river import metrics

model = tree.HoeffdingTreeClassifier()

metric = metrics.ClassificationReport()

for sample, target in dataset:
    prediction = model.predict_one(sample)
    if prediction is not None:# la primera iteración la predicción será None y no puede emplearse para actualizar
        metric.update(target, prediction)
    model.learn_one(sample, target)

print(metric)

[ClassificationReport](https://riverml.xyz/latest/api/metrics/ClassificationReport/)  recupera el *accuracy*, el *recall* y el *F1-Score* para cada clase que el modelo ha visto. Además, la columna *Support* indica el número de instancias identificadas en el flujo.  Este ejemplo muestra un *pipeline* tan frecuente que River ha encapsulado todo el proceso en una sola función: `progresive_val_score`.


In [ ]:
from river import evaluate

model = tree.HoeffdingTreeClassifier()
metric = metrics.ClassificationReport()

print(evaluate.progressive_val_score(dataset, model, metric))

### Ejercicio 4

**Emplea un modelo basado en los k vecinos más próximos para la clasificación multiclase**

River proporciona modelos basados en los 'k' vecinos más próximos para la clasificación multiclase. Revisa los modelos disponibles y prueba el apropiado. Configura el proceso de evaluación para imprimir las métricas cada 1.000 observaciones.

Este es un proceso que consume tiempo y recursos debido al tipo de modelo basado. Puedes detenerlo una vez que revises los primeros resultados de las métricas.

In [ ]:
#Emplea un modelo basado en los k vecinos más próximos para la clasificación multiclase





## 7. Aprendizaje online supervisado: Regresión


Por último,  veremos un ejemplo de regresión aplicado al aprendizaje incremental. Para este tipo de problemas, el modelo debe predecir una salida numérica dada una observación particular que representa la evolución de una serie temporal.

Una observación de regresión consiste en varias características y una etiqueta, que generalmente se codifica como un número continuo (aunque también puede ser discreto). Un dataset útil, incluido en la biblioteca River, es el [dataset](https://riverml.xyz/latest/api/datasets/TrumpApproval/) de la tasa de aprobación de Trump.



In [ ]:
from river import datasets

dataset = datasets.TrumpApproval()
print(dataset)

Como se ve arriba, cada observación tiene 6 características que se utilizan para hacer una predicción en el rango $[0, 1]$. Para este problema, utilizaremos un modelo de regresión. En particular, usaremos un [KNN](https://riverml.xyz/0.14.0/api/neighbors/KNNRegressor/)  adaptado que está incluido en River.

**Nota:** Ten en cuenta que los modelos de regresión no tienen el método `predict_proba_one()`, ya que no calculan probabilidades de clase.




In [ ]:
from river import neighbors

data_stream = iter(dataset)
sample, target = next(data_stream)

model = neighbors.KNNRegressor()
print("Predicción del modelo sin entrenar:", model.predict_one(sample))

Como se puede ver, el modelo aún no ha sido entrenado y, por lo tanto, la salida predeterminada es $0.0$. Ahora, vamos a entrenar el modelo y repetir la predicción para ver cómo varía.

In [ ]:
model.learn_one(sample, target)
print("Predicción del modelo  entrenado con una sola observación:", model.predict_one(sample))

Desarrollaremos ahora el proceso de una forma más completa empleando la **validación progresiva**, tal y cómo se hizo en los casos anteriores, aplicando la secuencia de operaciones: predicción, evaluación y entrenamiento.

In [ ]:
from river import metrics

model = neighbors.KNNRegressor()

metric = metrics.MAE()

for sample, target in dataset:
    prediction = model.predict_one(sample)
    metric.update(target, prediction)
    model.learn_one(sample, target)

print(metric)

Lo mismo en notación compacta (recomendada):

In [ ]:
from river import evaluate

model = neighbors.KNNRegressor()
metric = metrics.MAE()

evaluate.progressive_val_score(dataset, model, metric)

### Ejercicio 5
Es importante destacar que los modelos que dependen de métricas de distancia son altamente sensibles a las variaciones en las escalas de las características. **Establece un *pipeline* de preprocesamiento que incluya la operación de estandarización y reevalúa el modelo**.

In [ ]:
#Desarrolla un pipeline con estandarización y el modelo KNN




